In [1]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
from isolated_ad_model.processing import PREPROCESS
import logging 
from AnomalyModelFactory import AnomalyModelFactory
from AnomalyRobustnessEvaluator import AnomalyRobustnessEvaluator

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s [%(levelname)s] %(message)s', 
    handlers=[logging.StreamHandler()]
)

logger = logging.getLogger(__name__)

/home/wysoc/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wysoc/.local/lib/python3.8/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [4]:
#config
use_cached_model = True
model_name = "isolation_forest"
model_path = "model_" + model_name + ".pkl"


#obtain the dataset
dataset = pd.read_csv('../isolated_ad_model/ue.csv')
true_anomalies = dataset['Viavi.UE.anomalies']
ps = PREPROCESS(dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
ps.process()
training_data = ps.data

#obtain the model
model_factory = AnomalyModelFactory() # TODO: Add configuration from file

if use_cached_model:
    model = model_factory.load_model(model_path) 
else:
    model = model_factory.build_model(training_data, true_anomalies, model_name)
    model_factory.save_model(model_path)

AnomalyRobustnessEvaluator._save_generated_dataset(training_data, './ue_base_processed')

#testing enviroment
evaluator = AnomalyRobustnessEvaluator(model, training_data, true_anomalies)

In [3]:
evaluator.test_a2pm(save_adv_data=True)
evaluator.test_hsja(save_adv_data=True)

2025-04-08 14:08:32,395 [INFO] 
--- Before A2PM Attack ---
2025-04-08 14:08:32,396 [INFO] Inliers: 5700, Outliers: 4300
2025-04-08 14:08:32,411 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.74      0.84      7432
           1       0.56      0.93      0.70      2568

    accuracy                           0.79     10000
   macro avg       0.76      0.84      0.77     10000
weighted avg       0.86      0.79      0.80     10000

2025-04-08 14:08:32,420 [INFO] Macro F1: 0.7687
2025-04-08 14:08:32,421 [INFO] Confusion Matrix:
[[5523 1909]
 [ 177 2391]]
2025-04-08 14:08:32,427 [INFO] 
--- After A2PM Attack ---
2025-04-08 14:08:32,428 [INFO] Inliers: 26, Outliers: 9974
2025-04-08 14:08:32,444 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      7432
           1       0.25      0.99      0.41      2568

    accuracy                           0

In [ ]:
# base_dataset = pd.read_csv('./ue_base_processed')
# a2pm_dataseet = pd.read_csv('./ue_a2pm')

# base_dataset.compare(a2pm_dataseet)

du-id           RRU.PrbUsedDl           DRB.UEThpDl                \
          self     other          self     other        self         other   
0     0.377292  0.711710      0.028679  0.176190    0.000112  1.503205e-04   
1     0.571136  0.713464      0.033060  0.077927         NaN           NaN   
2     0.657619  0.582122      0.036717  0.054355    0.000066  5.542120e-07   
3          NaN       NaN           NaN       NaN    0.000053  1.624674e-04   
4          NaN       NaN           NaN       NaN    0.000050  1.316448e-05   
...        ...       ...           ...       ...         ...           ...   
9995       NaN       NaN      0.022584  0.057436         NaN           NaN   
9996  0.583076  0.683810      0.052954  0.041555    0.000086  1.665840e-04   
9997  0.784646  0.684183      0.048358  0.043117    0.000195  2.529334e-04   
9998  0.407620  0.465746      0.018108  0.098571    0.000075  1.763272e-04   
9999  0.701432  0.590024           NaN       NaN         NaN           NaN   

             x                   y           RF.serving.RSRQ            \
          self     other      self     other            self     other   
0     0.845835  0.280195 -0.375802 -0.617430       -0.004020 -0.034401   
1    -0.575696 -0.202399       NaN       NaN       -0.023252 -0.013619   
2          NaN       NaN       NaN       NaN       -0.026746 -0.031413   
3          NaN       NaN -0.629190 -0.782190       -0.028113 -0.022772   
4    -0.746271 -0.354791 -0.429671 -0.633746             NaN       NaN   
...        ...       ...       ...       ...             ...       ...   
9995       NaN       NaN -0.483790 -0.322583             NaN       NaN   
9996 -0.577839 -0.005109 -0.568063 -0.609070       -0.023738 -0.026152   
9997 -0.190923  0.024577  0.587176 -0.043057       -0.008419 -0.010243   
9998  0.819184  0.290869 -0.402792 -0.782190       -0.004343 -0.033022   
9999 -0.399018 -0.765592 -0.588727 -0.782190       -0.028556 -0.037100   

     RF.serving.RSSINR            
                  self     other  
0             0.012186  0.001187  
1                  NaN       NaN  
2                  NaN       NaN  
3                  NaN       NaN  
4             0.007109 -0.007575  
...                ...       ...  
9995          0.005946 -0.007575  
9996          0.007804 -0.005074  
9997          0.026346 -0.007575  
9998          0.013976  0.016488  
9999          0.011709 -0.007575  

[10000 rows x 14 columns]